In [8]:
pip install pandas numpy scikit-learn nltk matplotlib

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
import random
import hashlib  
import csv
import os
from datetime import datetime
from IPython.display import display, Markdown
import matplotlib.pyplot as plt

In [10]:
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [11]:
class ExamEvaluator:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.tfidf_vectorizer = TfidfVectorizer(stop_words='english')
        self.lemmatizer = WordNetLemmatizer()
        self.short_answer_threshold = 0.7
        self.min_answer_length = 3

    def preprocess_text(self, text):
        if not isinstance(text, str):
            return ""
        try:
            text = text.lower()
            text = re.sub(f'[{string.punctuation}]', ' ', text)
            text = re.sub(r'\s+', ' ', text).strip()
            return text
        except:
            return ""

    def evaluate_one_word(self, correct_answer, student_answer):
        correct_answer = str(correct_answer).strip().lower()
        student_answer = str(student_answer).strip().lower()
        correct_answer = re.sub(f'[{string.punctuation}]', '', correct_answer)
        student_answer = re.sub(f'[{string.punctuation}]', '', student_answer)
        correct_lemma = self.lemmatizer.lemmatize(correct_answer)
        student_lemma = self.lemmatizer.lemmatize(student_answer)
        if student_answer == correct_answer or student_lemma == correct_lemma:
            return 1.0, "Correct"
        if self.levenshtein_distance(correct_answer, student_answer) <= 1:
            return 0.5, "Partially correct (minor spelling error)"
        return 0.0, f"Incorrect. The correct answer is: {correct_answer}"

    def levenshtein_distance(self, s1, s2):
        if len(s1) < len(s2):
            return self.levenshtein_distance(s2, s1)
        if len(s2) == 0:
            return len(s1)
        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row
        return previous_row[-1]

    def evaluate_short_answer(self, correct_answer, student_answer):
        if not isinstance(correct_answer, str) or not isinstance(student_answer, str):
            return 0.0, "Invalid answer format"
        if len(student_answer.strip()) < self.min_answer_length:
            return 0.0, "Answer too short to evaluate"
        processed_correct = self.preprocess_text(correct_answer)
        processed_student = self.preprocess_text(student_answer)
        if not processed_student:
            return 0.0, "No answer provided."
        try:
            if not processed_correct or not processed_student:
                return 0.0, "Could not evaluate answer"
            tfidf_matrix = self.tfidf_vectorizer.fit_transform([processed_correct, processed_student])
            similarity_score = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        except ValueError:
            similarity_score = self.fallback_similarity(processed_correct, processed_student)
        except:
            similarity_score = 0.0
        feedback = self.generate_feedback(processed_correct, processed_student, similarity_score)
        if similarity_score >= 0.8:  
            numeric_score = 2.0
        elif similarity_score >= 0.6:   
            numeric_score = 1.5  
        elif similarity_score >= 0.4:  
            numeric_score = 1.0
        elif similarity_score >= 0.2:  
            numeric_score = 0.5
        else:                         
            numeric_score = 0.0
        similarity_score = round(similarity_score, 2)
        return numeric_score, feedback

    def fallback_similarity(self, text1, text2):
        tokens1 = set(word_tokenize(text1))
        tokens2 = set(word_tokenize(text2))
        tokens1 = {w for w in tokens1 if w not in self.stop_words}
        tokens2 = {w for w in tokens2 if w not in self.stop_words}
        if not tokens1 or not tokens2:
            return 0.0
        intersection = tokens1.intersection(tokens2)
        union = tokens1.union(tokens2)
        return len(intersection) / len(union)

    def generate_feedback(self, correct_answer, student_answer, similarity_score):
        if similarity_score >= self.short_answer_threshold:
            return "Answer is correct and complete."
        correct_tokens = word_tokenize(correct_answer)
        student_tokens = word_tokenize(student_answer)
        correct_keywords = [w for w in correct_tokens if w not in self.stop_words and len(w) > 2]
        student_keywords = [w for w in student_tokens if w not in self.stop_words and len(w) > 2]
        missing_keywords = [word for word in correct_keywords if word not in student_keywords]
        if missing_keywords:
            return f"Missing important concepts: {', '.join(missing_keywords[:5])}" + ("..." if len(missing_keywords) > 5 else "")
        else:
            return "The answer lacks detail or accuracy. Review the topic again."

    def evaluate_exam(self, questions_df):
        results = []
        for _, row in questions_df.iterrows():
            question = row.get('Question', '')
            q_type = row.get('Type', '').lower()
            correct_answer = row.get('Correct Answer', '')
            student_answer = row.get('Student Answer', '')
            if q_type == 'one-word' or q_type == 'one word':
                score, feedback = self.evaluate_one_word(correct_answer, student_answer)
                max_score = 1.0
            else:
                score, feedback = self.evaluate_short_answer(correct_answer, student_answer)
                max_score = 2.0
            results.append({
                'Question': question,
                'Type': q_type,
                'Correct Answer': correct_answer,
                'Student Answer': student_answer,
                'Score': score,
                'Max Score': max_score,
                'Feedback': feedback
            })
        return pd.DataFrame(results)

In [12]:
class QuestionBank:
    def __init__(self):
        self.questions_df = None
        self.question_types = ['one-word', 'short-answer']

    def validate_question_bank(self, df):
        if df.empty:
            raise ValueError("Question bank is empty")
        required_columns = {'Question', 'Type', 'Correct Answer'}
        missing_cols = required_columns - set(df.columns)
        if missing_cols:
            raise ValueError(f"Missing required columns: {', '.join(missing_cols)}")
        if df['Question'].isnull().any():
            raise ValueError("Some questions are missing")
        if df['Correct Answer'].isnull().any():
            raise ValueError("Some correct answers are missing")
        valid_types = {'one-word', 'short-answer'}
        invalid_types = set(df['Type'].unique()) - valid_types
        if invalid_types:
            raise ValueError(f"Invalid question types: {', '.join(invalid_types)}")

    def load_question_bank(self, filepath):
        try:
            if not os.path.exists(filepath):
                raise FileNotFoundError(f"File not found: {filepath}")
            df = pd.read_csv(filepath)
            self.validate_question_bank(df)
            type_mapping = {
                'one word': 'one-word',
                'short answer': 'short-answer'
            }
            df['Type'] = df['Type'].str.lower().replace(type_mapping)
            self.questions_df = df
            return True, f"Successfully loaded {len(df)} questions"
        except Exception as e:
            return False, f"Error loading question bank: {str(e)}"

    def get_random_exam(self, student_id, one_word_count=10, short_answer_count=5, seed=None):
        if self.questions_df is None:
            return None, "Question bank not loaded"
        if not isinstance(student_id, (str, int)):
            return None, "Invalid student ID"
        try:
            one_word_count = max(1, int(one_word_count))
            short_answer_count = max(1, int(short_answer_count))
        except:
            return None, "Invalid question counts"
        if seed is None:
            try:
                seed_str = str(student_id) + datetime.now().strftime("%Y%m%d")
                seed = int(hashlib.md5(seed_str.encode()).hexdigest(), 16) % (10**8)
            except:
                seed = random.randint(1, 10**8)
        random.seed(seed)
        one_word_questions = self.questions_df[self.questions_df['Type'] == 'one-word'].copy()
        short_answer_questions = self.questions_df[self.questions_df['Type'] == 'short-answer'].copy()
        if len(one_word_questions) < one_word_count:
            return None, f"Not enough one-word questions (have {len(one_word_questions)}, need {one_word_count})"
        if len(short_answer_questions) < short_answer_count:
            return None, f"Not enough short-answer questions (have {len(short_answer_questions)}, need {short_answer_count})"
        try:
            selected_one_word = one_word_questions.sample(n=one_word_count, replace=False)
            selected_short_answer = short_answer_questions.sample(n=short_answer_count, replace=False)
            exam_questions = pd.concat([selected_one_word, selected_short_answer])
            exam_questions = exam_questions.sample(frac=1).reset_index(drop=True)
            exam_questions['Student Answer'] = ""
            exam_data = {
                'student_id': student_id,
                'timestamp': datetime.now().isoformat(),
                'exam_seed': seed,
                'one_word_count': one_word_count,
                'short_answer_count': short_answer_count,
                'total_questions': len(exam_questions)
            }
            return exam_questions, exam_data
        except:
            return None, "Error generating exam"

    def create_sample_question_bank(self, filepath):
        sample_data = {
            'Question': [
                'What is the capital of France?',
                'What is the chemical symbol for gold?',
                'What is the largest planet in our solar system?',
                'Who wrote "Romeo and Juliet"?',
                'What is the square root of 144?',
                'What is the symbol for the element oxygen?',
                'What is the powerhouse of the cell?',
                'What is the largest ocean on Earth?',
                'What is the smallest prime number?',
                'Who painted the Mona Lisa?',
                'What is the hardest natural material?',
                'Which planet is closest to the Sun?',
                'What is the currency of Japan?',
                'Who discovered penicillin?',
                'What is the chemical formula for water?',
                'Define photosynthesis.',
                'Explain the process of cellular respiration.',
                'What are the key components of object-oriented programming?',
                'Describe Newton\'s First Law of Motion.',
                'Explain the concept of supply and demand in economics.',
                'What is the significance of DNA in genetics?',
                'Define the theory of relativity.',
                'Explain the water cycle.',
                'Describe the process of mitosis.',
                'What is the significance of the Declaration of Independence?'
            ],
            'Type': ['one-word'] * 15 + ['short-answer'] * 10,
            'Correct Answer': [
                'Paris',
                'Au',
                'Jupiter',
                'Shakespeare',
                '12',
                'O',
                'Mitochondria',
                'Pacific',
                '2',
                'DaVinci',
                'Diamond',
                'Mercury',
                'Yen',
                'Fleming',
                'H2O',
                'Photosynthesis is the process by which green plants and some other organisms convert light energy into chemical energy. Plants use sunlight, water, and carbon dioxide to create oxygen and energy in the form of sugar.',
                'Cellular respiration is the process by which cells convert nutrients into ATP, carbon dioxide, and water. It is essentially the reverse of photosynthesis where oxygen is used to break down nutrients to release energy.',
                'Object-oriented programming is based on four main principles: encapsulation (hiding data within a class), inheritance (creating new classes from existing ones), polymorphism (different implementations for the same interface), and abstraction (simplifying complex systems).',
                'Newton\'s First Law of Motion states that an object at rest stays at rest, and an object in motion stays in motion with the same speed and direction unless acted upon by an unbalanced force. This principle is also known as the law of inertia.',
                'Supply and demand is an economic model that explains how prices are determined in a market. When supply exceeds demand, prices fall. When demand exceeds supply, prices rise. Market equilibrium occurs when supply equals demand.',
                'DNA (deoxyribonucleic acid) is the hereditary material in humans and almost all other organisms. It contains the genetic instructions used in the development and functioning of all known living organisms.',
                'Einstein\'s theory of relativity describes how space and time are linked for objects moving at a consistent speed in a straight line. The theory has two components: special relativity and general relativity, which introduced the concept that massive objects cause a distortion in space-time.',
                'The water cycle is the continuous movement of water within the Earth and atmosphere. It includes processes such as evaporation, condensation, precipitation, infiltration, runoff, and transpiration.',
                'Mitosis is a type of cell division that results in two daughter cells each having the same number and kind of chromosomes as the parent nucleus. It consists of prophase, metaphase, anaphase, and telophase, followed by cytokinesis.',
                'The Declaration of Independence, adopted on July 4, 1776, announced the separation of 13 North American British colonies from Great Britain. It explained the philosophy of self-governance and outlined grievances against King George III.'
            ]
        }
        df = pd.DataFrame(sample_data)
        df.to_csv(filepath, index=False)
        return filepath

In [13]:
class ExamSystemCLI:
    def __init__(self):
        self.evaluator = ExamEvaluator()
        self.question_bank = QuestionBank()
        self.current_student_id = None
        self.exam_df = None
        self.results_df = None
        self.current_mode = None
        self.student_history = {}
        self.performance_analytics = {}

    def clear_screen(self):
        from IPython.display import clear_output
        clear_output()

    def display_header(self, title):
        display(Markdown(f"## {title}"))
        print("-" * 50)

    def professor_mode(self):
        self.current_mode = 'professor'
        while True:
            self.clear_screen()
            self.display_header("Professor Mode")
            print("1. Load Question Bank")
            print("2. Create Sample Question Bank")
            print("3. View Current Question Bank")
            print("4. Set Evaluation Threshold (Current: {:.1f})".format(self.evaluator.short_answer_threshold))
            print("5. Switch to Student Mode")
            print("6. Exit")
            choice = input("\nEnter your choice (1-6): ").strip()
            if choice == "1":
                self.load_question_bank()
            elif choice == "2":
                self.create_sample_bank()
            elif choice == "3":
                self.view_question_bank()
            elif choice == "4":
                self.set_threshold()
            elif choice == "5":
                self.student_mode()
                return
            elif choice == "6":
                print("Exiting the system...")
                return
            else:
                print("Invalid choice. Please try again.")
                input("Press Enter to continue...")

    def student_mode(self):
        self.current_mode = 'student'
        while True:
            self.clear_screen()
            self.display_header("Student Mode")
            print("1. Take New Exam")
            print("2. View Previous Results")
            print("3. Performance Analytics")
            print("4. Review Question Bank")
            print("5. Practice Mode")
            print("6. Switch to Professor Mode")
            print("7. Exit")
            choice = input("\nEnter your choice (1-7): ").strip()
            if choice == "1":
                self.take_exam()
            elif choice == "2":
                self.view_previous_results()
            elif choice == "3":
                self.view_performance_analytics()
            elif choice == "4":
                self.review_question_bank()
            elif choice == "5":
                self.practice_mode()
            elif choice == "6":
                self.professor_mode()
                return
            elif choice == "7":
                print("Exiting the system...")
                return
            else:
                print("Invalid choice. Please try again.")
                input("Press Enter to continue...")

    def load_student_history(self, student_id):
        if student_id not in self.student_history:
            self.student_history[student_id] = []
        results_dir = "exam_results"
        if not os.path.exists(results_dir):
            return
        pattern = f"student_{student_id}_*.csv"
        result_files = [f for f in os.listdir(results_dir) if f.startswith(f"student_{student_id}_")]
        for filename in result_files:
            filepath = os.path.join(results_dir, filename)
            try:
                if not any(f['filepath'] == filepath for f in self.student_history[student_id]):
                    df = pd.read_csv(filepath)
                    timestamp = filename.split('_')[2].replace('.csv', '')
                    self.student_history[student_id].append({
                        'filepath': filepath,
                        'timestamp': timestamp,
                        'results': df
                    })
            except:
                pass

    def load_question_bank(self):
        self.clear_screen()
        self.display_header("Load Question Bank")
        filepath = input("Enter path to question bank CSV file: ").strip()
        if not filepath:
            print("No file path provided.")
            input("Press Enter to continue...")
            return
        success, message = self.question_bank.load_question_bank(filepath)
        if success:
            print(f"Success: {message}")
        else:
            print(f"Error: {message}")
        input("Press Enter to continue...")

    def create_sample_bank(self):
        self.clear_screen()
        self.display_header("Create Sample Question Bank")
        filepath = input("Enter path to save sample question bank (e.g., sample_bank.csv): ").strip()
        if not filepath:
            print("No file path provided.")
            input("Press Enter to continue...")
            return
        if not filepath.lower().endswith('.csv'):
            filepath += '.csv'
        try:
            saved_path = self.question_bank.create_sample_question_bank(filepath)
            print(f"Sample question bank created at: {saved_path}")
            success, message = self.question_bank.load_question_bank(saved_path)
            if success:
                print(f"Successfully loaded the sample question bank.")
            else:
                print(f"Error loading the sample bank: {message}")
        except:
            print("Error creating sample question bank")
        input("Press Enter to continue...")

    def view_question_bank(self):
        self.clear_screen()
        self.display_header("Question Bank Preview")
        if self.question_bank.questions_df is None or self.question_bank.questions_df.empty:
            print("No question bank loaded.")
            input("Press Enter to continue...")
            return
        display(self.question_bank.questions_df)
        input("\nPress Enter to continue...")

    def set_threshold(self):
        self.clear_screen()
        self.display_header("Set Evaluation Threshold")
        print(f"Current threshold: {self.evaluator.short_answer_threshold}")
        print("This threshold determines how similar a student's answer must be to be considered correct (0-1).")
        try:
            new_threshold = float(input("Enter new threshold (0-1): "))
            if 0 <= new_threshold <= 1:
                self.evaluator.short_answer_threshold = new_threshold
                print(f"Threshold set to {new_threshold}")
            else:
                print("Threshold must be between 0 and 1. No changes made.")
        except:
            print("Invalid input. Please enter a number between 0 and 1.")
        input("Press Enter to continue...")

    def take_exam(self):
        self.clear_screen()
        self.display_header("Take Exam")
    
        if self.question_bank.questions_df is None or self.question_bank.questions_df.empty:
            print("No question bank loaded. Please ask your professor to load one.")
            input("Press Enter to continue...")
            return
    
        student_id = input("Enter your student ID: ").strip()
        if not student_id:
            print("Student ID is required.")
            input("Press Enter to continue...")
            return
    
        self.current_student_id = student_id
        exam_questions, exam_data = self.question_bank.get_random_exam(student_id)
        if exam_questions is None:
            print(f"Error: {exam_data}")
            input("Press Enter to continue...")
            return
    
        self.exam_df = exam_questions.copy()
        self.exam_df['Student Answer'] = ""
    
        print(f"\nExam for Student ID: {student_id}")
        print(f"Contains {exam_data['one_word_count']} one-word questions (1 mark each)")
        print(f"and {exam_data['short_answer_count']} short-answer questions (2 marks each)")
        print("\nAnswer all questions to the best of your ability.")
        input("\nPress Enter to begin the exam...")
    
        # Simplified question loop without review/change option
        for i, (_, row) in enumerate(self.exam_df.iterrows()):
            self.clear_screen()
            self.display_header(f"Question {i+1} of {len(self.exam_df)}")
            print(f"Type: {row['Type'].capitalize()}")
            print(f"Marks: {'1' if row['Type'] == 'one-word' else '2'}")
            print(f"\nQuestion: {row['Question']}")
    
            if row['Type'] == 'one-word':
                answer = input("\nYour answer (one word): ").strip()
            else:
                print("\nYour answer (short answer, multiple sentences allowed):")
                answer = input("> ").strip()
    
            self.exam_df.at[i, 'Student Answer'] = answer
    
            # Only show "Next question" or "Submit" for all but last question
            if i < len(self.exam_df) - 1:
                input("\nPress Enter to continue to next question...")
    
        self.clear_screen()
        print("Submitting your exam...")
        self.results_df = self.evaluator.evaluate_exam(self.exam_df)
        self.save_results()
        self.load_student_history(self.current_student_id)
        self.update_performance_analytics()
    
        print("\nExam submitted successfully!")
        print("\n1. View results now")
        print("2. Return to menu")
        choice = input("\nChoose (1-2): ")
        if choice == "1":
            self.view_results()

    def view_previous_results(self):
        self.clear_screen()
        self.display_header("Previous Exam Results")

        pd.set_option('display.max_colwidth', 200)  

        if not self.current_student_id:
            print("No student ID set. Please take an exam first.")
            input("Press Enter to continue...")
            return

        self.load_student_history(self.current_student_id)

        if not self.student_history.get(self.current_student_id):
            print("No previous exam results found.")
            input("Press Enter to continue...")
            return

        print(f"Exam History for Student ID: {self.current_student_id}")
        print("\nAvailable Exams:")
        for i, exam in enumerate(self.student_history[self.current_student_id], 1):
            print(f"{i}. {exam['timestamp']} - {len(exam['results'])} questions")

        exam_choice = input("\nSelect exam to view (or '0' to cancel): ")
        if exam_choice == "0":
            return

        try:
            exam_idx = int(exam_choice) - 1
            selected_exam = self.student_history[self.current_student_id][exam_idx]

            self.clear_screen()
            self.display_header(f"Exam Results - {selected_exam['timestamp']}")

            results_df = selected_exam['results']
            total_score = results_df['Score'].sum()
            max_score = results_df['Max Score'].sum()
            percentage = (total_score / max_score) * 100

            print(f"Date: {selected_exam['timestamp']}")
            print(f"Total Score: {total_score:.1f}/{max_score:.1f} ({percentage:.1f}%)")

            print("\nQuestion Details:")
            with pd.option_context('display.max_colwidth', 200):  
                display(results_df[['Question', 'Type', 'Score', 'Max Score', 'Feedback']])

            print("\n1. Review specific question")
            print("2. Return to menu")
            sub_choice = input("\nChoose (1-2): ")
            if sub_choice == "1":
                q_num = input("Enter question number to review: ")
                try:
                    q_idx = int(q_num) - 1
                    if 0 <= q_idx < len(results_df):
                        self.review_question(results_df.iloc[q_idx])
                except:
                    print("Invalid question number.")
        except:
            print("Invalid selection.")

        input("\nPress Enter to continue...")
        pd.reset_option('display.max_colwidth')

    def review_question(self, question_row):
        self.clear_screen()
        self.display_header("Question Review")
        print(f"Question: {question_row['Question']}")
        print(f"Type: {question_row['Type'].capitalize()}")
        print(f"\nYour Answer: {question_row['Student Answer']}")
        print(f"\nCorrect Answer: {question_row['Correct Answer']}")
        print(f"\nScore: {question_row['Score']}/{question_row['Max Score']}")
        print(f"Feedback: {question_row['Feedback']}")
        input("\nPress Enter to continue...")

    def update_performance_analytics(self):
        if not self.current_student_id:
            return
        self.load_student_history(self.current_student_id)
        if not self.student_history.get(self.current_student_id):
            return
        if self.current_student_id not in self.performance_analytics:
            self.performance_analytics[self.current_student_id] = {
                'attempts': 0,
                'average_score': 0,
                'question_types': {},
                'weak_areas': [],
                'improvement_over_time': []
            }
        analytics = self.performance_analytics[self.current_student_id]
        analytics['attempts'] = len(self.student_history[self.current_student_id])
        total_score = 0
        total_max = 0
        for exam in self.student_history[self.current_student_id]:
            df = exam['results']
            total_score += df['Score'].sum()
            total_max += df['Max Score'].sum()
        analytics['average_score'] = (total_score / total_max) * 100 if total_max > 0 else 0
        type_stats = {}
        for exam in self.student_history[self.current_student_id]:
            for _, row in exam['results'].iterrows():
                q_type = row['Type']
                if q_type not in type_stats:
                    type_stats[q_type] = {'correct': 0, 'total': 0}
                type_stats[q_type]['total'] += 1
                if row['Score'] >= row['Max Score'] * 0.8:
                    type_stats[q_type]['correct'] += 1
        analytics['question_types'] = {
            q_type: {
                'accuracy': stats['correct'] / stats['total'] * 100 if stats['total'] > 0 else 0,
                'count': stats['total']
            }
            for q_type, stats in type_stats.items()
        }
        all_questions = []
        for exam in self.student_history[self.current_student_id]:
            for _, row in exam['results'].iterrows():
                all_questions.append({
                    'question': row['Question'],
                    'score': row['Score'],
                    'max': row['Max Score'],
                    'percentage': (row['Score'] / row['Max Score']) * 100 if row['Max Score'] > 0 else 0
                })
        weak_questions = sorted(all_questions, key=lambda x: x['percentage'])[:5]
        analytics['weak_areas'] = weak_questions
        improvement = []
        for exam in self.student_history[self.current_student_id]:
            df = exam['results']
            total = df['Score'].sum()
            max_total = df['Max Score'].sum()
            percentage = (total / max_total) * 100 if max_total > 0 else 0
            improvement.append({
                'timestamp': exam['timestamp'],
                'score': percentage
            })
        analytics['improvement_over_time'] = sorted(improvement, key=lambda x: x['timestamp'])

    def view_performance_analytics(self):
        self.clear_screen()
        self.display_header("Performance Analytics")
        if not self.current_student_id:
            print("No student ID set. Please take an exam first.")
            input("Press Enter to continue...")
            return
        self.update_performance_analytics()
        analytics = self.performance_analytics.get(self.current_student_id, {})
        if not analytics:
            print("No analytics data available yet.")
            input("Press Enter to continue...")
            return
        print(f"Performance Summary for Student ID: {self.current_student_id}")
        print(f"\nTotal Exams Taken: {analytics.get('attempts', 0)}")
        print(f"Average Score: {analytics.get('average_score', 0):.1f}%")
        print("\nPerformance by Question Type:")
        for q_type, stats in analytics.get('question_types', {}).items():
            print(f"- {q_type.capitalize()}: {stats['accuracy']:.1f}% accuracy ({stats['count']} questions)")
        weak_areas = analytics.get('weak_areas', [])
        if weak_areas:
            print("\nWeakest Areas (need improvement):")
            for i, area in enumerate(weak_areas, 1):
                print(f"{i}. {area['question']} ({area['percentage']:.1f}%)")
        improvement = analytics.get('improvement_over_time', [])
        if len(improvement) > 1:
            print("\nProgress Over Time:")
            dates = [x['timestamp'] for x in improvement]
            scores = [x['score'] for x in improvement]
            try:
                plt.figure(figsize=(10, 4))
                plt.plot(dates, scores, marker='o')
                plt.title("Exam Performance Over Time")
                plt.xlabel("Exam Date")
                plt.ylabel("Score (%)")
                plt.ylim(0, 100)
                plt.grid(True)
                plt.show()
            except:
                pass
        input("\nPress Enter to continue...")

    def review_question_bank(self):
        self.clear_screen()
        self.display_header("Question Bank Review")
        if self.question_bank.questions_df is None or self.question_bank.questions_df.empty:
            print("No question bank loaded.")
            input("Press Enter to continue...")
            return
        print("Available Question Types:")
        print("1. One-word questions")
        print("2. Short-answer questions")
        print("3. All questions")
        print("4. Search questions")
        print("5. Back to menu")
        choice = input("\nSelect option (1-5): ")
        if choice == "1":
            df = self.question_bank.questions_df[self.question_bank.questions_df['Type'] == 'one-word']
            print("\nOne-word Questions:")
            display(df[['Question', 'Correct Answer']])
        elif choice == "2":
            df = self.question_bank.questions_df[self.question_bank.questions_df['Type'] == 'short-answer']
            print("\nShort-answer Questions:")
            display(df[['Question', 'Correct Answer']])
        elif choice == "3":
            print("\nAll Questions:")
            display(self.question_bank.questions_df[['Question', 'Type', 'Correct Answer']])
        elif choice == "4":
            search_term = input("Enter search term: ").lower()
            mask = self.question_bank.questions_df['Question'].str.lower().str.contains(search_term)
            results = self.question_bank.questions_df[mask]
            print(f"\nFound {len(results)} matching questions:")
            display(results[['Question', 'Type', 'Correct Answer']])
        elif choice == "5":
            return
        else:
            print("Invalid choice.")
        input("\nPress Enter to continue...")

    def practice_mode(self):
        self.clear_screen()
        self.display_header("Practice Mode")
        if self.question_bank.questions_df is None or self.question_bank.questions_df.empty:
            print("No question bank loaded.")
            input("Press Enter to continue...")
            return
        print("Practice Mode Options:")
        print("1. Practice specific question type")
        print("2. Practice weak areas (requires completed exams)")
        print("3. Random practice questions")
        print("4. Back to menu")
        choice = input("\nSelect option (1-4): ")
        if choice == "1":
            print("\nSelect question type:")
            print("1. One-word questions")
            print("2. Short-answer questions")
            type_choice = input("Choose (1-2): ")
            if type_choice == "1":
                q_type = 'one-word'
            elif type_choice == "2":
                q_type = 'short-answer'
            else:
                print("Invalid choice.")
                input("Press Enter to continue...")
                return
            questions = self.question_bank.questions_df[self.question_bank.questions_df['Type'] == q_type]
            self.run_practice_session(questions)
        elif choice == "2":
            if not self.current_student_id or not self.performance_analytics.get(self.current_student_id):
                print("No exam history available to identify weak areas.")
                input("Press Enter to continue...")
                return
            weak_questions = []
            for item in self.performance_analytics[self.current_student_id]['weak_areas']:
                matches = self.question_bank.questions_df[
                    self.question_bank.questions_df['Question'] == item['question']
                ]
                if not matches.empty:
                    weak_questions.append(matches.iloc[0])
            if not weak_questions:
                print("No weak area questions found in current question bank.")
                input("Press Enter to continue...")
                return
            print(f"\nPracticing your {len(weak_questions)} weakest areas:")
            self.run_practice_session(pd.DataFrame(weak_questions))
        elif choice == "3":
            num_questions = input("How many random questions? (default 5): ")
            try:
                num_questions = int(num_questions) if num_questions else 5
            except:
                num_questions = 5
            if num_questions > len(self.question_bank.questions_df):
                num_questions = len(self.question_bank.questions_df)
            questions = self.question_bank.questions_df.sample(n=num_questions)
            self.run_practice_session(questions)
        elif choice == "4":
            return
        else:
            print("Invalid choice.")
        input("\nPress Enter to continue...")

    def run_practice_session(self, questions_df):
        score = 0
        max_score = 0
        for i, (_, row) in enumerate(questions_df.iterrows()):
            self.clear_screen()
            print(f"Practice Question {i+1} of {len(questions_df)}")
            print(f"Type: {row['Type'].capitalize()}")
            print(f"\nQuestion: {row['Question']}")
            if row['Type'] == 'one-word':
                answer = input("\nYour answer (one word): ").strip()
            else:
                print("\nYour answer (short answer, multiple sentences allowed):")
                answer = input("> ").strip()
            if row['Type'] == 'one-word':
                points, feedback = self.evaluator.evaluate_one_word(row['Correct Answer'], answer)
                max_points = 1.0
            else:
                points, feedback = self.evaluator.evaluate_short_answer(row['Correct Answer'], answer)
                max_points = 2.0

            score += points
            max_score += max_points

            print("\nFeedback:")
            print(f"Your answer: {answer}")
            print(f"Correct answer: {row['Correct Answer']}")
            print(f"Score: {points}/{max_points}")
            print(feedback)

            input("\nPress Enter to continue...")

        self.clear_screen()
        print("Practice Session Complete!")
        print(f"\nTotal Score: {score:.1f}/{max_score:.1f}")
        print(f"Percentage: {(score/max_score)*100:.1f}%")

        if max_score > 0 and score/max_score < 0.7:
            print("\nRecommendation: You might want to review these topics again.")
        elif max_score > 0 and score/max_score >= 0.9:
            print("\nExcellent work! You've mastered these questions.")

    def view_results(self):
        self.clear_screen()
        self.display_header("Exam Results")
        
        if self.results_df is None or self.results_df.empty:
            print("No exam results available.")
            input("Press Enter to continue...")
            return
        
        total_score = self.results_df['Score'].sum()
        max_possible = self.results_df['Max Score'].sum()
        percentage = (total_score / max_possible) * 100
        
        one_word_count = self.results_df[self.results_df['Type'] == 'one-word'].shape[0]
        short_answer_count = self.results_df[self.results_df['Type'] == 'short-answer'].shape[0]
        
        one_word_score = self.results_df[self.results_df['Type'] == 'one-word']['Score'].sum()
        one_word_max = one_word_count * 1.0
        
        short_answer_score = self.results_df[self.results_df['Type'] == 'short-answer']['Score'].sum()
        short_answer_max = short_answer_count * 2.0
        
        print(f"Student ID: {self.current_student_id}")
        print(f"Total Score: {total_score:.1f}/{max_possible:.1f} ({percentage:.1f}%)")
        print(f"One-Word Questions: {one_word_score:.1f}/{one_word_max:.1f}")
        print(f"Short-Answer Questions: {short_answer_score:.1f}/{short_answer_max:.1f}")
        
        print("\nDetailed Results:")
        with pd.option_context('display.max_colwidth', 200):  # Context manager for wider columns
            display(self.results_df[['Question', 'Type', 'Score', 'Max Score', 'Feedback']])
        
        input("\nPress Enter to continue...")

    def save_results(self):
        if self.results_df is None or self.results_df.empty or not self.current_student_id:
            return

        results_dir = "exam_results"
        if not os.path.exists(results_dir):
            os.makedirs(results_dir)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{results_dir}/student_{self.current_student_id}_{timestamp}.csv"

        try:
            self.results_df.to_csv(filename, index=False)
        except Exception as e:
            print(f"Warning: Could not save results to file: {str(e)}")

In [14]:
def main():
    system = ExamSystemCLI()
    print("Starting in Professor Mode...")
    system.professor_mode()

if __name__ == "__main__":
    main()

## Student Mode

--------------------------------------------------
1. Take New Exam
2. View Previous Results
3. Performance Analytics
4. Review Question Bank
5. Practice Mode
6. Switch to Professor Mode
7. Exit
Exiting the system...
